In [7]:
import pandas as pd
import numpy as np
import tsfresh
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf

In [8]:
import os

import matplotlib.pyplot as plt

# keras goodies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K

In [9]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import ConvLSTM2D

In [10]:
data_directory = os.path.join(os.getcwd(), 'Kai_Respeck_data')
filenames = []

for root, _, files in os.walk(data_directory, topdown=False):
    for name in files:
        filenames.append(os.path.join(root, name))

set(filenames)

{'/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Climbing_stairs.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Descending_stairs.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Desk_work.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Falling_on_the_back.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Falling_on_the_knees.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Falling_on_the_left.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Falling_on_the_right.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Lying_down_left.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Lying_down_on_back.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Practical/Kai_Respeck_data/Lying_down_on_right.csv',
 '/afs/inf.ed.ac.uk/user/s18/s1894401/PDIoT/Pract

In [11]:
dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))

In [12]:
Kai_df = pd.concat(dataframes).reset_index(drop=True)
Kai_df

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,sensor_type,activity_type,activity_code,subject_id,notes,recording_id
0,1632922547206,0.015137,-0.638733,-0.106506,6.609375,2.500000,3.171875,Respeck,Climbing stairs,12,s1800883,NaN,Respeck_s1800883_Climbing stairs_29-09-2021_14...
1,1632922547235,0.036621,-0.818176,-0.220764,3.687500,-4.812500,1.453125,Respeck,Climbing stairs,12,s1800883,NaN,Respeck_s1800883_Climbing stairs_29-09-2021_14...
2,1632922547281,-0.022949,-1.087952,-0.211731,-1.609375,-5.156250,-10.078125,Respeck,Climbing stairs,12,s1800883,NaN,Respeck_s1800883_Climbing stairs_29-09-2021_14...
3,1632922547327,0.006592,-1.313293,-0.159973,-8.921875,-0.218750,-11.781250,Respeck,Climbing stairs,12,s1800883,NaN,Respeck_s1800883_Climbing stairs_29-09-2021_14...
4,1632922547356,0.017090,-1.298401,-0.305969,-19.750000,-6.578125,-5.171875,Respeck,Climbing stairs,12,s1800883,NaN,Respeck_s1800883_Climbing stairs_29-09-2021_14...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12925,1632919138876,0.109863,0.042908,-0.953674,-0.453125,0.171875,0.421875,Respeck,Lying down on stomach,8,s1800883,NaN,Respeck_s1800883_Lying down on stomach_29-09-2...
12926,1632919138907,0.110596,0.035583,-0.950012,-0.328125,0.078125,0.484375,Respeck,Lying down on stomach,8,s1800883,NaN,Respeck_s1800883_Lying down on stomach_29-09-2...
12927,1632919138952,0.110352,0.041199,-0.953186,-0.421875,0.046875,0.296875,Respeck,Lying down on stomach,8,s1800883,NaN,Respeck_s1800883_Lying down on stomach_29-09-2...
12928,1632919138998,0.110107,0.038025,-0.954163,-0.437500,-0.140625,0.406250,Respeck,Lying down on stomach,8,s1800883,NaN,Respeck_s1800883_Lying down on stomach_29-09-2...


In [13]:
base_df = pd.read_csv('Respeck_recordings_clean.csv')
movement_indices = np.asarray(base_df[base_df.activity_type == 'Movement'].index) # drop general movement
base_df.drop(movement_indices, inplace=True)
base_df.reset_index(drop=True, inplace=True)

/afs/inf.ed.ac.uk/user/s18/s1894401/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
# cleaning data
df_respeck = base_df[(base_df['activity_type']=='Standing') & (base_df['accel_y'] < -0.5)]
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Walking at normal speed') & (base_df['accel_y'] < 0.5)]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Climbing stairs') & (base_df['accel_y'] < 0)]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Desk work') & (base_df['accel_y'] < -0.5)]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Sitting') & (base_df['accel_y'] < -0.5)]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Sitting bent forward') & (base_df['accel_y'] < -0.25)]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Sitting bent backward') & (base_df['accel_y'] < 0)]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Lying down on back')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Lying down on stomach')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Lying down left')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Lying down right')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Falling on the left')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Falling on the right')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Falling on knees')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Falling on the back')]])
df_respeck = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Running')]])
base_df = pd.concat([df_respeck, base_df[(base_df['activity_type']=='Descending stairs') & (base_df['accel_x'] > -0.5) & (base_df['accel_y'] < 0.5)]])
base_df.reset_index(inplace=True)

In [15]:
columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

In [16]:
falling_dfs = []
for _ in range(10):
    falling_dfs.append(base_df[base_df.activity_type == 'Falling on knees'])
    falling_dfs.append(base_df[base_df.activity_type == 'Falling on the right'])
    falling_dfs.append(base_df[base_df.activity_type == 'Falling on the left'])
    falling_dfs.append(base_df[base_df.activity_type == 'Falling on the back'])

falling_df = pd.concat(falling_dfs).reset_index(drop=True) # oversample for falling

running_dfs = []
for _ in range(2):
    running_dfs.append(base_df[base_df.activity_type == 'Running']) 

running_df = pd.concat(running_dfs).reset_index(drop=True) # oversample for running

In [17]:
base_df = pd.concat([base_df, falling_df, running_df]).reset_index(drop=True)

In [18]:
nan_indices = set()
for i in range(len(base_df)):
    for col in columns_of_interest:
        if pd.isnull(base_df.loc[i, col]):
            nan_indices.add(i)

In [19]:
base_df.drop(list(nan_indices), inplace=True)

In [20]:
subject_dataframes = []
for rid, group in base_df.groupby("subject_id"):
    subject_dataframes.append(group)
n = len(subject_dataframes)
n

46

In [21]:
# Get the final sliding windows for the whole dataset

window_size = 50 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
step_size = 25 # this is 50% overlap

window_number = 0 # start a counter at 0 to keep track of the window number

all_overlapping_windows = []

for rid, group in base_df.groupby("recording_id"):
    # print(f"Processing rid = {rid}")
    
    large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]
    
    overlapping_windows = large_enough_windows[::step_size] 
    
    # then we will append a window ID to each window
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1
    
    if len(overlapping_windows) == 0:
            continue
    
    all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))


final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)
subject_ids = list(set(base_df['subject_id']))
training_partitions = [final_sliding_windows[final_sliding_windows.subject_id != s] for s in subject_ids]
testing_partitions = [final_sliding_windows[final_sliding_windows.subject_id == s] for s in subject_ids]

/afs/inf.ed.ac.uk/user/s18/s1894401/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/afs/inf.ed.ac.uk/user/s18/s1894401/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [22]:
# Get the final sliding windows for Kai's dataset

window_size = 50 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
step_size = 25 # this is 50% overlap

window_number = 0 # start a counter at 0 to keep track of the window number

all_overlapping_windows = []

for rid, group in Kai_df.groupby("recording_id"):
    # print(f"Processing rid = {rid}")
    
    large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]
    
    overlapping_windows = large_enough_windows[::step_size] 
    
    # then we will append a window ID to each window
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1
    
    if len(overlapping_windows) == 0:
            continue
    
    all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))


Kai_final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)

In [23]:
print(f"The data was collected using the sensors: {base_df.sensor_type.unique()}")
print(f"The data was collected for the activities: {base_df.activity_type.unique()}")
print(f"The number of unique recordings is: {len(base_df.recording_id.unique())}")
print(f"The subject IDs in the recordings are: {len(base_df.subject_id.unique())}")

The data was collected using the sensors: ['Respeck']
The data was collected for the activities: ['Standing' 'Walking at normal speed' 'Climbing stairs' 'Desk work'
 'Sitting' 'Sitting bent forward' 'Sitting bent backward'
 'Lying down on back' 'Lying down on stomach' 'Lying down left'
 'Lying down right' 'Falling on the left' 'Falling on the right'
 'Falling on knees' 'Falling on the back' 'Running' 'Descending stairs']
The number of unique recordings is: 809
The subject IDs in the recordings are: 46


In [24]:
reduced_class_labels = {
    'Falling on the left':0,
    'Falling on knees':0,
    'Falling on the back':0,
    'Descending stairs':3,
    'Standing':1,
    'Lying down right':2,
    'Walking at normal speed':3,
    'Lying down on back':2,
    'Desk work':1,
    'Running':3,
    'Climbing stairs':3,
    'Falling on the right':0,
    'Sitting bent backward':1,
    'Sitting bent forward':1,
    'Lying down left':2,
    'Lying down on stomach':2,
    'Sitting':1
}

In [25]:
class_labels = {
    'Falling on the left':0,
    'Falling on knees':1,
    'Falling on the back':2,
    'Descending stairs':3,
    'Standing':4,
    'Lying down right':5,
    'Walking at normal speed':6,
    'Lying down on back':7,
    'Desk work':8,
    'Running':9,
    'Climbing stairs':10,
    'Falling on the right':11,
    'Sitting bent backward':12,
    'Sitting bent forward':13,
    'Lying down left':14,
    'Lying down on stomach':16,
    'Sitting':17
}

In [26]:
falling_class_labels = {
    0:0,
    11:1,
    2:2,
    1:3
}

falling_class_labels_inv = {
    0:0,
    1:11,
    2:2,
    3:1
}

In [27]:
sitting_standing_class_labels = {
    13:0,
    4:1,
    17:2,
    8:3,
    12:4
}

sitting_standing_class_labels_inv = {
    0:13,
    1:4,
    2:17,
    3:8,
    4:12
}

In [28]:
lying_class_labels = {
    16:0,
    7:1,
    14:2,
    5:3
}

lying_class_labels_inv = {
    0:16,
    1:7,
    2:14,
    3:5
}

In [29]:
moving_class_labels = {
    10:0,
    3:1,
    6:2,
    9:3
}

moving_class_labels_inv = {
    0:10,
    1:3,
    2:6,
    3:9
}

In [30]:
len(final_sliding_windows)

1163900

In [31]:
n

46

In [32]:
# randomly select five partitions and train the model on them
random_partitions = [4, 16, 27, 31, 40] # 16 corresponds to the partition that excludes Kai's data

In [33]:
# get the training and testing features
feature_lists_train = []
feature_lists_test = []

In [34]:
for i in random_partitions:
    print(i)
    # now extract all features
    feature_list = []

    for col in columns_of_interest:
        new_features = tsfresh.extract_features(timeseries_container=training_partitions[i], column_id='window_id',
                            column_value=col, default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters(), n_jobs=8)
        feature_list.append(new_features)

    feature_list = pd.concat(feature_list, axis=1)
    
    feature_lists_train.append(feature_list)

4


Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 10.56it/s]


16


Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 10.56it/s]


27


Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 10.44it/s]


31


Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 10.61it/s]


40


Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 10.27it/s]


In [35]:
for i in random_partitions:
    print(i)
    # now extract all features
    feature_list = []

    for col in columns_of_interest:
        new_features = tsfresh.extract_features(timeseries_container=testing_partitions[i], column_id='window_id',
                            column_value=col, default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters(), n_jobs=8)
        feature_list.append(new_features)

    feature_list = pd.concat(feature_list, axis=1)
    
    feature_lists_test.append(feature_list)

4


Feature Extraction: 100%|██████████| 40/40 [00:00<00:00, 375.89it/s]

16



Feature Extraction: 100%|██████████| 39/39 [00:00<00:00, 295.91it/s]


27


Feature Extraction: 100%|██████████| 37/37 [00:00<00:00, 350.21it/s]

31



Feature Extraction: 100%|██████████| 39/39 [00:00<00:00, 402.07it/s]

40



Feature Extraction: 100%|██████████| 38/38 [00:00<00:00, 381.19it/s]


In [36]:
filters = 64
kernel_size = 3
n_features = 6
activation='relu'
n_classes = 4
window_size

50

In [37]:
X_train = []
y_train = []
y_train_reduced = []

for window_id, group in training_partitions[16].groupby('window_id'): # use partition 16 so that Kai's data is excluded from training set
    # print(f"window_id = {window_id}")
    shape = group[columns_of_interest].values.shape
    # print(f"shape = {shape}")    
    X_train.append(group[columns_of_interest].values)
    y_train.append(class_labels[group["activity_type"].values[0]])
    y_train_reduced.append(reduced_class_labels[group["activity_type"].values[0]])
    
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
y_train_reduced = np.asarray(y_train_reduced)

print(f"X train shape = {X_train.shape}")
print(f"y train shape = {y_train.shape}")

X train shape = (22699, 50, 6)
y train shape = (22699,)


In [38]:
falling_indices = [j for j in range(len(y_train)) if (y_train[j]==0 or y_train[j]==1 or y_train[j]==2 or y_train[j]==11)]
X_train_falling  = np.asarray([X_train[j] for j in falling_indices])
y_train_falling = np.asarray([y_train[j] for j in falling_indices])


In [39]:
sitting_standing_indices = [j for j in range(len(y_train)) if (y_train[j]==4 or y_train[j]==8 or y_train[j]==12 or y_train[j]==13 or y_train[j]==17)]
X_train_sitting_standing = np.asarray([X_train[j] for j in sitting_standing_indices])
y_train_sitting_standing = np.asarray([y_train[j] for j in sitting_standing_indices])


In [40]:
lying_indices = [j for j in range(len(y_train)) if (y_train[j]==5 or y_train[j]==7 or y_train[j]==14 or y_train[j]==16)]
X_train_lying = np.asarray([X_train[j] for j in lying_indices])
y_train_lying = np.asarray([y_train[j] for j in lying_indices])


In [41]:
moving_indices = [j for j in range(len(y_train)) if (y_train[j]==3 or y_train[j]==6 or y_train[j]==9 or y_train[j]==10 or y_train[j]==15)]
X_train_moving = np.asarray([X_train[j] for j in moving_indices])
y_train_moving = np.asarray([y_train[j] for j in moving_indices])


In [42]:
print(len(falling_indices))
print(len(sitting_standing_indices))
print(len(lying_indices))
print(len(moving_indices))

4361
5924
5231
7183


In [43]:
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17}

In [44]:
y_train_falling = np.asarray([falling_class_labels[y_train_falling[j]] for j in range(len(y_train_falling))])

y_train_sitting_standing = np.asarray([sitting_standing_class_labels[y_train_sitting_standing[j]] for j in range(len(y_train_sitting_standing))])

y_train_lying = np.asarray([lying_class_labels[y_train_lying[j]] for j in range(len(y_train_lying))])

y_train_moving = np.asarray([moving_class_labels[y_train_moving[j]] for j in range(len(y_train_moving))])

In [45]:
# Use Kai's data for testing

X_test = []
y_test = []
y_test_reduced = []


for window_id, group in Kai_final_sliding_windows.groupby('window_id'):
    # print(f"window_id = {window_id}")
    shape = group[columns_of_interest].values.shape
    # print(f"shape = {shape}")    
    X_test.append(group[columns_of_interest].values)
    y_test.append(class_labels[group["activity_type"].values[0]])
    y_test_reduced.append(reduced_class_labels[group["activity_type"].values[0]])

X_test = np.array(X_test)
y_test = np.array(y_test)
y_test_reduced = np.array(y_test_reduced)

print(f"X test shape = {X_test.shape}")
print(f"y test shape = {y_test.shape}")

X test shape = (431, 50, 6)
y test shape = (431,)


In [46]:
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps, n_length = 2, 25
X_train = X_train.reshape((X_train.shape[0], n_steps, 1, n_length, n_features))
X_test = X_test.reshape((X_test.shape[0], n_steps, 1, n_length, n_features))

# the ConvLSTM Network Model for the top hierarchy

model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 1, 23, 64)         54016     
_________________________________________________________________
dropout (Dropout)            (None, 1, 23, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1472)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               147300    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 201,720
Trainable params: 201,720
Non-trainable params: 0
_________________________________________________________________


In [47]:
n_classes

4

In [48]:
def OHE(num, n_classes):
    return ([0 for x in range(num)] + [1] + [0 for x in range(n_classes - num - 1)])

In [49]:
def get_OHE(l, n_classes):
    return ([OHE(x, n_classes) for x in l])

In [50]:
y_train_reduced = get_OHE(y_train_reduced, n_classes) # get one-hot-encoding

In [51]:
y_train_reduced = np.asarray(y_train_reduced, dtype=float)

In [52]:
print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train_reduced.shape}")

X_train shape = (22699, 2, 1, 25, 6)
y_train shape = (22699, 4)


In [53]:
# fit networkf or top layer
model.fit(X_train, y_train_reduced, epochs=5, batch_size=128)

Epoch 1/5
178/178 [==============================] - 21s 105ms/step - loss: 0.9075 - accuracy: 0.6809
Epoch 2/5
178/178 [==============================] - 11s 63ms/step - loss: 0.0856 - accuracy: 0.9751
Epoch 3/5
178/178 [==============================] - 11s 62ms/step - loss: 0.0559 - accuracy: 0.9827
Epoch 4/5
178/178 [==============================] - 22s 121ms/step - loss: 0.0411 - accuracy: 0.9866
Epoch 5/5
178/178 [==============================] - 26s 145ms/step - loss: 0.0386 - accuracy: 0.9883


In [54]:
# stats
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_pred_labels.shape

(431,)

In [55]:
accuracy = accuracy_score(y_test_reduced, y_pred_labels)
accuracy

1.0

In [56]:
X_train_fallings = [X_train_falling for _ in range(5)]
X_train_falling_concat = np.concatenate(X_train_fallings)
y_train_fallings = [y_train_falling for _ in range(5)]
y_train_falling_concat = np.concatenate(y_train_fallings)

X_train_sitting_standings = [X_train_sitting_standing for _ in range(5)]
X_train_sitting_standing_concat = np.concatenate(X_train_sitting_standings)
y_train_sitting_standings = [y_train_sitting_standing for _ in range(5)]
y_train_sitting_standing_concat = np.concatenate(y_train_sitting_standings)

X_train_lyings = [X_train_lying for _ in range(5)]
X_train_lying_concat = np.concatenate(X_train_lyings)
y_train_lyings = [y_train_lying for _ in range(5)]
y_train_lying_concat = np.concatenate(y_train_lyings)

X_train_movings = [X_train_moving for _ in range(5)]
X_train_moving_concat = np.concatenate(X_train_movings)
y_train_movings = [y_train_moving for _ in range(5)]
y_train_moving_concat = np.concatenate(y_train_movings)

In [57]:
pred_falling_indices = [j for j in range(len(y_pred_labels)) if (y_pred_labels[j]==0)]
pred_sitting_standing_indices = [j for j in range(len(y_pred_labels)) if (y_pred_labels[j]==1)]
pred_lying_indices = [j for j in range(len(y_pred_labels)) if (y_pred_labels[j]==2)]
pred_moving_indices = [j for j in range(len(y_pred_labels)) if (y_pred_labels[j]==3)]

In [58]:
# reconstruct the original labels
y_pred_orig_labels = np.zeros(shape=(len(y_pred_labels),), dtype=int)

In [59]:
X_pred_falling = np.array([X_test[j] for j in pred_falling_indices])
X_pred_falling.shape

(54, 2, 1, 25, 6)

In [60]:
X_pred_sitting_standing = np.array([X_test[j] for j in pred_sitting_standing_indices])
X_pred_sitting_standing.shape

(145, 2, 1, 25, 6)

In [61]:
X_pred_lying = np.array([X_test[j] for j in pred_lying_indices])
X_pred_lying.shape

(116, 2, 1, 25, 6)

In [62]:
X_pred_moving = np.array([X_test[j] for j in pred_moving_indices])
X_pred_moving.shape

(116, 2, 1, 25, 6)

In [63]:
# develop model for distinguishing lying types

def predict_lying_category(X_test):
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]*X_test.shape[3], X_test.shape[4])
    X_test_means = np.mean(X_test, axis=1)
    y_pred = np.zeros(shape=(X_test.shape[0],), dtype=int)
    for i in range(X_test_means.shape[0]):
        accel_x, accel_y, accel_z = X_test_means[i][0], X_test_means[i][1], X_test_means[i][2]
        accel_norm = np.sqrt(accel_x**2 + accel_y**2 + accel_z**2)
        cos_theta_z = accel_z/accel_norm
        theta_z = np.arccos(cos_theta_z) * 180/np.pi # get angle in degrees
        if theta_z >= 0.0 and theta_z <= 45.0:
            y_pred[i] = 1 # on back
        elif theta_z > 45.0 and theta_z <= 90.0:
            y_pred[i] = 3 # on right
        elif theta_z > 90.0 and theta_z <= 135.0:
            y_pred[i] = 2 # on left
        elif theta_z > 135.0 and theta_z <= 180.0:
            y_pred[i] = 0 # on stomach
        else:
            y_pred[i] = 1 # default is on back
    return y_pred


In [64]:
y_pred_lying_labels = predict_lying_category(X_pred_lying)

In [70]:
# convert labels back to original
y_pred_lying_orig_labels = [lying_class_labels_inv[y_pred_lying_labels[j]] for j in range(len(y_pred_lying_labels))]
for j in range(len(y_pred_lying_labels)):
    y_pred_orig_labels[pred_lying_indices[j]] = y_pred_lying_orig_labels[j]

In [66]:
def setup_falling_model(n_classes=4, X_train_falling=X_train_falling_concat, y_train_falling=y_train_falling_concat):
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length = 2, 25
    X_train_falling = X_train_falling.reshape((X_train_falling.shape[0], n_steps, 1, n_length, n_features))
    
    # the ConvLSTM Network Model for the 'falling' class
    falling_model = Sequential()
    falling_model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    falling_model.add(Dropout(0.5))
    falling_model.add(Flatten())
    falling_model.add(Dense(100, activation='relu'))
    falling_model.add(Dense(n_classes, activation='softmax'))
    falling_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    y_train_falling = np.asarray(get_OHE(y_train_falling, n_classes), dtype=np.float32)
    
    falling_model.fit(X_train_falling, y_train_falling, epochs=5, batch_size=64)
    return falling_model

In [67]:
falling_model = setup_falling_model()

if len(X_pred_falling != 0):
    y_pred_falling_ohe = falling_model.predict(X_pred_falling)
    y_pred_falling_labels = np.argmax(y_pred_falling_ohe, axis=1)
    # convert labels back to original
    y_pred_falling_orig_labels = [falling_class_labels_inv[y_pred_falling_labels[j]] for j in range(len(y_pred_falling_labels))]
    # print(np.asarray(y_pred_falling_orig_labels))
    for j in range(len(y_pred_falling_labels)):
        y_pred_orig_labels[pred_falling_indices[j]] = y_pred_falling_orig_labels[j]

Epoch 1/5
341/341 [==============================] - 11s 27ms/step - loss: 0.5592 - accuracy: 0.8829
Epoch 2/5
341/341 [==============================] - 14s 40ms/step - loss: 0.0103 - accuracy: 0.9974
Epoch 3/5
341/341 [==============================] - 28s 83ms/step - loss: 0.0036 - accuracy: 0.9991
Epoch 4/5
341/341 [==============================] - 20s 60ms/step - loss: 0.0032 - accuracy: 0.9989
Epoch 5/5
341/341 [==============================] - 9s 26ms/step - loss: 0.0039 - accuracy: 0.9988


In [68]:
y_pred_orig_labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16

In [69]:
y_test

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16